In [1]:
import gym
import matplotlib.pyplot as plt
import numpy as np

In [2]:
import sys
sys.path.append("../")

In [3]:
%load_ext autoreload
%autoreload 2

In [4]:
from mbt_gym.gym.TradingEnvironment import TradingEnvironment
from mbt_gym.gym.Traders import TradinghWithSpeedTrader

In [5]:
from mbt_gym.agents.BaselineAgents import CarteaJaimungalMmAgent
from mbt_gym.gym.helpers.generate_trajectory import generate_trajectory
from mbt_gym.gym.StableBaselinesTradingEnvironment import StableBaselinesTradingEnvironment
from mbt_gym.gym.wrappers import *
from mbt_gym.rewards.RewardFunctions import CjCriterion, CjMmCriterion
from mbt_gym.stochastic_processes.midprice_models import *
from mbt_gym.stochastic_processes.price_impact_models import *

In [6]:
terminal_time = 1.0
temporary_impact = 1e-3
permanent_impact = 1e-3
n_steps = int(100)
phi = 0.5
alpha = 0.001
num_trajectories = 3

In [7]:
sigma = 0.001
initial_inventory = (-4,5)
initial_price = 100
step_size = terminal_time/n_steps
midprice_model = BrownianMotionMidpriceModel(volatility=sigma, 
                                           terminal_time=terminal_time, 
                                           step_size=step_size, 
                                           initial_price=initial_price, 
                                           num_trajectories=num_trajectories)
price_impact_model = TemporaryAndPermanentPriceImpact(temporary_impact_coefficient = temporary_impact,
                            permanent_impact_coefficient = permanent_impact,
                            n_steps = n_steps,
                            terminal_time = terminal_time,
                            num_trajectories = num_trajectories)
Speedtrader = TradinghWithSpeedTrader(midprice_model = midprice_model, price_impact_model = price_impact_model, 
                            num_trajectories = num_trajectories)

In [8]:
Speedtrader.price_impact_model

In [9]:
def get_cj_env_OE(num_trajectories:int = 1):    
    sigma = 0.001
    initial_inventory = (-4,5)
    initial_price = 100
    step_size = terminal_time/n_steps
    midprice_model = BrownianMotionMidpriceModel(volatility=sigma, 
                                                   terminal_time=terminal_time, 
                                                   step_size=step_size, 
                                                   initial_price=initial_price, 
                                                   num_trajectories=num_trajectories)
    price_impact_model = TemporaryAndPermanentPriceImpact(temporary_impact_coefficient = temporary_impact,
                                permanent_impact_coefficient = permanent_impact,
                                n_steps = n_steps,
                                terminal_time = terminal_time,
                                num_trajectories = num_trajectories)
    Speedtrader = TradinghWithSpeedTrader(midprice_model = midprice_model, price_impact_model = price_impact_model, 
                                num_trajectories = num_trajectories)
   
    env_params = dict(terminal_time=terminal_time, 
                      n_steps=n_steps,
                      initial_inventory = initial_inventory,
                      midprice_model = midprice_model,
                      price_impact_model = price_impact_model,
                      trader = Speedtrader,
                      reward_function = CjMmCriterion(phi, alpha),
                      max_inventory=n_steps,
                      normalise_action_space = False,
                      normalise_observation_space = False,
                      num_trajectories=num_trajectories)
    return TradingEnvironment(**env_params)

In [10]:
env = get_cj_env_OE(num_trajectories=num_trajectories)

In [11]:
env.reset()

array([[  0.,   4.,   0., 100.,   0.],
       [  0.,   0.,   0., 100.,   0.],
       [  0.,   3.,   0., 100.,   0.]])

CASH_INDEX = 0
INVENTORY_INDEX = 1
TIME_INDEX = 2
ASSET_PRICE_INDEX = 3

In [12]:
env.state

array([[  0.,   4.,   0., 100.,   0.],
       [  0.,   0.,   0., 100.,   0.],
       [  0.,   3.,   0., 100.,   0.]])

In [13]:
env.trader.price_impact_model

In [14]:
env.trader.price_impact_model.temporary_impact_coefficient

0.001

In [15]:
action = np.array([[0.0], 
                    [1.0], 
                     [-10]])
env.step(action = action)

(array([[ 0.00000000e+00,  4.00000000e+00,  1.00000000e-02,
          1.00000119e+02,  0.00000000e+00],
        [-1.00001000e+00,  1.00000000e-02,  1.00000000e-02,
          9.99998929e+01,  1.00000000e-05],
        [ 9.99900000e+00,  2.90000000e+00,  1.00000000e-02,
          9.99999712e+01, -1.00000000e-04]]),
 array([-7.96824546e-02, -1.16712792e-05, -4.26334953e-02]),
 array([False, False, False]),
 [{}, {}, {}])